In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Necessary Packages

In [ ]:
!pip install git+https://github.com/tensorflow/docs
!pip install -q transformers
!pip install patool
!pip install timm 
!pip install pillow

Importing Libraries and Modules

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os 
#from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths
import patoolib
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
import timm
import sys
from tqdm import tqdm
import time
import copy
from PIL import Image
from timm.loss import LabelSmoothingCrossEntropy
import matplotlib.pyplot as plt
%matplotlib inline




['abseiling', 'air drumming']


Configuration

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Number of GPUs available:", torch.cuda.device_count())

Unzipping the folder

In [ ]:
patoolib.extract_archive("/content/drive/MyDrive/Frame Dataset.zip")

Classes Available

In [ ]:
dataset_path = os.listdir(r'/content/drive/MyDrive/Dataset/Dataset')

label_types = os.listdir(r'/content/drive/MyDrive/Dataset/Dataset')
print (label_types)

## Data Preparation

Frame Extraction Main

In [ ]:
import os
import shutil
import imageio
import cv2
# Set up paths
Sequence_length=5
IMAGE_HEIGHT , IMAGE_WIDTH= 224, 224
data_dir = "/content/drive/MyDrive/Dataset/Dataset"
frames_dir = "/content/drive/MyDrive/Dataset/Model_Save"

# Get class names and their corresponding directories
classes_dir = data_dir
class_names = os.listdir(classes_dir)
class_dirs = [os.path.join(classes_dir, c) for c in class_names]

# Loop over each class directory
for i, class_dir in enumerate(class_dirs):
    # Create a directory to store the frames
    frames_class_dir = os.path.join(frames_dir, class_names[i])
    os.makedirs(frames_class_dir, exist_ok=True)

    # Loop over each video file in the class directory
    for video_file in os.listdir(class_dir):
        video_path = os.path.join(class_dir, video_file)
        video_name = os.path.splitext(video_file)[0]

        # Create a directory to store the frames for this video
        frames_video_dir = os.path.join(frames_class_dir, video_name)
       # os.makedirs(frames_video_dir, exist_ok=True)

        # Read the video and extract its frames
        reader = cv2.VideoCapture(video_path)
        video_frames_count = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames_window = max(int(video_frames_count/Sequence_length), 1)
        
        for frame_counter in range(Sequence_length):
            
        # Set the current frame position of the video.
            reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

            # Reading the frame from the video. 
            success, frame = reader.read() 

            # Check if Video frame is not successfully read then break the loop
            if not success:
                break

            # Resize the Frame to fixed height and width.
            resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
            frame_path = os.path.join(frames_class_dir, f"{frame_counter * skip_frames_window}.jpg")
            imageio.imwrite(frame_path, resized_frame)
          
        reader.release()


print("Frames extraction completed successfully!")


Frames extraction completed successfully!


In [ ]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [ ]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
            T.RandomErasing(p=0.2, value='random')
        ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return val_loader, test_loader, len(val_data), len(test_data)

In [ ]:
dataset_path = "/content/Frame Dataset"

In [ ]:
(train_loader, train_data_len) = get_data_loaders(dataset_path, 128, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 128, train=False)

In [ ]:
classes = get_classes("/content/Frame Dataset/train")
print(classes, len(classes))

['abseiling', 'air drumming', 'answering questions', 'applauding', 'applying cream', 'archery', 'arm wrestling', 'arranging flowers', 'assembling computer', 'auctioning', 'baby waking up', 'baking cookies', 'balloon blowing', 'bandaging', 'barbequing', 'bartending', 'beatboxing', 'bee keeping', 'belly dancing', 'bench pressing', 'bending back', 'bending metal', 'biking through snow', 'blasting sand', 'blowing glass', 'blowing leaves', 'blowing nose', 'blowing out candles', 'bobsledding', 'bookbinding', 'bouncing on trampoline', 'bowling', 'braiding hair', 'breading or breadcrumbing', 'breakdancing', 'brush painting', 'brushing hair', 'brushing teeth', 'building cabinet', 'building shed', 'bungee jumping', 'busking', 'canoeing or kayaking', 'capoeira', 'carrying baby', 'cartwheeling', 'carving pumpkin', 'catching fish', 'catching or throwing baseball', 'catching or throwing frisbee', 'catching or throwing softball', 'celebrating', 'changing oil', 'changing wheel', 'checking tires', 'che

In [ ]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [ ]:
print(len(train_loader), len(val_loader))
print(train_data_len, valid_data_len)

399 76
50960 9727


### [Importing Deit Model]

In [ ]:
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main
Downloading: "https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth" to /root/.cache/torch/hub/checkpoints/deit_base_patch16_224-b5f2ef4d.pth
100%|██████████| 330M/330M [00:03<00:00, 95.2MB/s]


In [ ]:
for param in model.parameters(): #freeze model
    param.requires_grad = False

n_inputs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(n_inputs, 512),
    nn.ReLU(),
    #nn.GELU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(classes))
)
model = model.to(device)
print(model.head)

Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=400, bias=True)
)


In [ ]:
print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

## Setting HyperParameters

In [ ]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

## Training Process

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=18):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler) # now it is a lot faster
# I will come back after 18 epochs

## Model Testing

In [ ]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model.eval()

for data, target in tqdm(test_loader):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == 64:
        for i in range(64):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

test_loss = test_loss / test_data_len
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print("Test Accuracy of %5s: %2d%% (%2d/%2d)" % (
            classes[i], 100*class_correct[i]/class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))
print("Test Accuracy of %2d%% (%2d/%2d)" % (
            100*np.sum(class_correct)/np.sum(class_total), np.sum(class_correct), np.sum(class_total)
        ))

## Single Video Prediction

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.io as tio
from PIL import Image
import matplotlib.pyplot as plt

# Load the video from the internet or local path
video_path = r"C:\Users\PRANAV BHARDWAJ\Videos\video5" # Replace with the video URL or local path
video_frames, _, info = tio.read_video(video_path)

# Define the transformation to be applied to each frame
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize each frame to the desired input size
    transforms.ToTensor(),  # Convert the frame to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the frame
])

# Set the device for inference (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model
model = torch.load(r"C:\Users\PRANAV BHARDWAJ\Downloads\HAR(UCF-101).pt")  # Replace with the path to your trained model
model = model.to(device)
model.eval()

# Load the label.txt file
with open(r'C:\Users\PRANAV BHARDWAJ\PycharmProjects\pythonProject1\UCF.txt', 'r') as f:
    label_names = f.read().splitlines()

# Create a list to store the predicted labels as strings for each frame
predicted_labels = []

# Iterate through each frame and make predictions
for frame in video_frames:
    # Apply the transformation to the frame
    frame = transform(Image.fromarray(frame.numpy()))

    # Add a batch dimension to the frame
    frame = frame.unsqueeze(0)

    # Move the frame to the device
    frame = frame.to(device)

    # Perform the forward pass
    with torch.no_grad():
        output = model(frame)

    # Get the predicted label for the frame
    _, predicted = torch.max(output, 1)
    predicted_label = predicted.item()

    # Convert the predicted label index to label string
    predicted_label_str = label_names[predicted_label]
    predicted_labels.append(predicted_label_str)

# Visualize the predicted labels
plt.plot(predicted_labels)
plt.xlabel('Frame Index')
plt.ylabel('Predicted Label')
plt.show()


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.io as tio
from PIL import Image
import matplotlib.pyplot as plt
import statistics

# Load the video from the internet or local path
video_path = r"C:\Users\PRANAV BHARDWAJ\OneDrive\Desktop\HAR Dataset\HAR\test\Bowling\v_Bowling_g06_c05.avi" # Replace with the video URL or local path
video_frames, _, info = tio.read_video(video_path)

# Define the transformation to be applied to each frame
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize each frame to the desired input size
    transforms.ToTensor(),  # Convert the frame to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the frame
])

# Set the device for inference (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model
model = torch.load(r"C:\Users\PRANAV BHARDWAJ\Downloads\HAR(UCF-101).pt")  # Replace with the path to your trained model
model = model.to(device)
model.eval()

# Load the label.txt file
with open(r'C:\Users\PRANAV BHARDWAJ\PycharmProjects\pythonProject1\UCF.txt', 'r') as f:
    label_names = f.read().splitlines()

# Create a list to store the predicted labels as strings for each frame
predicted_labels = []

# Iterate through each frame and make predictions
for frame in video_frames:
    # Apply the transformation to the frame
    frame = transform(Image.fromarray(frame.numpy()))

    # Add a batch dimension to the frame
    frame = frame.unsqueeze(0)

    # Move the frame to the device
    frame = frame.to(device)

    # Perform the forward pass
    with torch.no_grad():
        output = model(frame)

    # Get the predicted label for the frame
    _, predicted = torch.max(output, 1)
    predicted_label = predicted.item()

    # Convert the predicted label index to label string
    predicted_label_str = label_names[predicted_label]
    predicted_labels.append(predicted_label_str)

# Compute the mode of the predicted labels
final_prediction = statistics.mode(predicted_labels)

print("Final Prediction:", final_prediction)


## Save the Trained Model

In [ ]:
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model.cpu(), example)
traced_script_module.save("HAR.pt")